In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda:0')

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [3]:
path = 'C:/Users/ideal/Downloads/jupyter/user_data/'

In [4]:
train_png = sorted(glob(path + 'train/*.png'))
test_png = sorted(glob(path + 'test/*.png'))

In [5]:
len(train_png), len(test_png)

(858, 215)

In [6]:
train_y = pd.read_csv(path +"train.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [7]:
label_unique

{'1': 0,
 '10-1': 1,
 '10-2': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [8]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
#     img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [9]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|███████████████████████████████████████████████████████████████████████████████| 215/215 [00:00<00:00, 757.70it/s]


In [10]:
np.save(path + 'train_imgs_384', np.array(train_imgs))
np.save(path + 'test_imgs_384', np.array(test_imgs))

In [11]:
train_imgs = np.load(path + 'train_imgs_384.npy')
test_imgs = np.load(path + 'test_imgs_384.npy')

In [12]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

train_meanR = np.mean([m[0] for m in meanRGB])/255
train_meanG = np.mean([m[1] for m in meanRGB])/255
train_meanB = np.mean([m[2] for m in meanRGB])/255

train_stdR = np.mean([s[0] for s in stdRGB])/255
train_stdG = np.mean([s[1] for s in stdRGB])/255
train_stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",train_meanR, train_meanG, train_meanB)
print("train 표준편차",train_stdR, train_stdG, train_stdB)

train 평균 0.587861452947292 0.5398018372012267 0.4853426659853918
train 표준편차 0.15058758283288234 0.15921522386293296 0.17031454681984776


In [13]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

test_meanR = np.mean([m[0] for m in meanRGB])/255
test_meanG = np.mean([m[1] for m in meanRGB])/255
test_meanB = np.mean([m[2] for m in meanRGB])/255

test_stdR = np.mean([s[0] for s in stdRGB])/255
test_stdG = np.mean([s[1] for s in stdRGB])/255
test_stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",test_meanR, test_meanG, test_meanB)
print("test 표준편차",test_stdR, test_stdG, test_stdB)

test 평균 0.5915704246815005 0.5468021681783177 0.49356994941872095
test 표준편차 0.15494109227168867 0.1642936360901455 0.1756015391157054


In [14]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [train_meanR, train_meanG, train_meanB],
                                     std = [train_stdR, train_stdG, train_stdB]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [test_meanR, test_meanG, test_meanB],
                                     std = [test_stdR, test_stdG, test_stdB])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('vgg16', pretrained=True, num_classes=11)
        if self.mode == 'test':
          self.model = timm.create_model('vgg16', pretrained=True, num_classes=11)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [15]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [16]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022, shuffle = True)
batch_size = 8
epochs = 100
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay = 1e-3)
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, path +'best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\ideal/.cache\torch\hub\checkpoints\vgg16-397923af.pth


-----------------SAVE:1 epoch----------------
epoch : 1/100    time : 13s/1271s
TRAIN    loss : 1.59183    f1 : 0.44606
Val    loss : 0.80320    f1 : 0.63795
-----------------SAVE:2 epoch----------------
epoch : 2/100    time : 11s/1032s
TRAIN    loss : 0.89282    f1 : 0.68176
Val    loss : 0.81969    f1 : 0.70291
-----------------SAVE:3 epoch----------------
epoch : 3/100    time : 11s/1034s
TRAIN    loss : 0.52956    f1 : 0.81460
Val    loss : 0.45741    f1 : 0.79324
-----------------SAVE:4 epoch----------------
epoch : 4/100    time : 11s/1013s
TRAIN    loss : 0.36732    f1 : 0.87948
Val    loss : 0.47753    f1 : 0.82827
epoch : 5/100    time : 8s/804s
TRAIN    loss : 0.35039    f1 : 0.88894
Val    loss : 0.55347    f1 : 0.80567
epoch : 6/100    time : 8s/778s
TRAIN    loss : 0.41957    f1 : 0.86775
Val    loss : 0.70938    f1 : 0.78008
epoch : 7/100    time : 8s/779s
TRAIN    loss : 0.29657    f1 : 0.90008
Val    loss : 0.48036    f1 : 0.81129
-----------------SAVE:8 epoch---------

epoch : 42/100    time : 9s/505s
TRAIN    loss : 0.08209    f1 : 0.97489
Val    loss : 0.17078    f1 : 0.95322
epoch : 43/100    time : 9s/495s
TRAIN    loss : 0.08867    f1 : 0.98070
Val    loss : 0.23982    f1 : 0.90221
epoch : 44/100    time : 9s/487s
TRAIN    loss : 0.06109    f1 : 0.97863
Val    loss : 0.13126    f1 : 0.94595
epoch : 45/100    time : 9s/478s
TRAIN    loss : 0.05608    f1 : 0.99004
Val    loss : 0.24507    f1 : 0.93667
epoch : 46/100    time : 9s/467s
TRAIN    loss : 0.07400    f1 : 0.98429
Val    loss : 0.10871    f1 : 0.95844
epoch : 47/100    time : 9s/459s
TRAIN    loss : 0.00713    f1 : 1.00000
Val    loss : 0.16007    f1 : 0.96407
-----------------SAVE:48 epoch----------------
epoch : 48/100    time : 11s/561s
TRAIN    loss : 0.01971    f1 : 0.99443
Val    loss : 0.12821    f1 : 0.97549
-----------------SAVE:49 epoch----------------
epoch : 49/100    time : 11s/564s
TRAIN    loss : 0.04090    f1 : 0.99437
Val    loss : 0.08481    f1 : 0.97568
epoch : 50/100  

epoch : 42/100    time : 8s/474s
TRAIN    loss : 0.03756    f1 : 0.98696
Val    loss : 0.27643    f1 : 0.93826
epoch : 43/100    time : 8s/469s
TRAIN    loss : 0.19522    f1 : 0.94858
Val    loss : 0.45633    f1 : 0.87325
epoch : 44/100    time : 8s/462s
TRAIN    loss : 0.07601    f1 : 0.97697
Val    loss : 0.16969    f1 : 0.94405
epoch : 45/100    time : 8s/449s
TRAIN    loss : 0.01474    f1 : 0.99427
Val    loss : 0.21240    f1 : 0.93751
epoch : 46/100    time : 8s/439s
TRAIN    loss : 0.09398    f1 : 0.97286
Val    loss : 0.35428    f1 : 0.90441
epoch : 47/100    time : 8s/433s
TRAIN    loss : 0.09185    f1 : 0.96415
Val    loss : 0.15786    f1 : 0.95483
epoch : 48/100    time : 8s/437s
TRAIN    loss : 0.01953    f1 : 0.99405
Val    loss : 0.35812    f1 : 0.88715
epoch : 49/100    time : 8s/414s
TRAIN    loss : 0.14504    f1 : 0.96285
Val    loss : 0.33582    f1 : 0.93822
epoch : 50/100    time : 8s/408s
TRAIN    loss : 0.10979    f1 : 0.97069
Val    loss : 0.53558    f1 : 0.83693
e

epoch : 51/100    time : 8s/407s
TRAIN    loss : 0.06984    f1 : 0.98037
Val    loss : 0.25781    f1 : 0.90595
epoch : 52/100    time : 8s/404s
TRAIN    loss : 0.18126    f1 : 0.95353
Val    loss : 0.25848    f1 : 0.92656
epoch : 53/100    time : 8s/393s
TRAIN    loss : 0.13580    f1 : 0.96353
Val    loss : 0.45107    f1 : 0.88288
epoch : 54/100    time : 8s/383s
TRAIN    loss : 0.16002    f1 : 0.95037
Val    loss : 0.24680    f1 : 0.93216
epoch : 55/100    time : 8s/376s
TRAIN    loss : 0.07505    f1 : 0.97851
Val    loss : 0.31500    f1 : 0.89549
epoch : 56/100    time : 8s/368s
TRAIN    loss : 0.06366    f1 : 0.98344
Val    loss : 0.13390    f1 : 0.96668
epoch : 57/100    time : 8s/361s
TRAIN    loss : 0.06098    f1 : 0.98512
Val    loss : 0.19789    f1 : 0.96605
epoch : 58/100    time : 8s/351s
TRAIN    loss : 0.04932    f1 : 0.98399
Val    loss : 0.30245    f1 : 0.91755
epoch : 59/100    time : 8s/342s
TRAIN    loss : 0.12816    f1 : 0.96870
Val    loss : 0.38878    f1 : 0.91057
e

epoch : 53/100    time : 8s/386s
TRAIN    loss : 0.06530    f1 : 0.98506
Val    loss : 0.23610    f1 : 0.93373
epoch : 54/100    time : 8s/380s
TRAIN    loss : 0.04271    f1 : 0.98285
Val    loss : 0.53503    f1 : 0.89276


In [17]:
pred_ensemble = []
batch_size = 8
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [18]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [19]:
label_decoder = {val:key for key, val in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

In [20]:
submission = pd.read_csv(path + "sample_submission.csv")
submission["label"] = f_result
submission

,file_name,label
0,001.png,1
1,002.png,2
2,003.png,1
3,004.png,6
4,005.png,8
...,...,...
210,211.png,5
211,212.png,8
212,213.png,3
213,214.png,6


In [21]:
submission.to_csv(path + "submit.csv", index = False)

In [22]:
# d9249@kyonggi.ac.kr

from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'C:/Users/ideal/Downloads/jupyter/user_data/submit.csv', 
'02438df9bd0f6300dae6ddea845e7e01d2cb1881849c166bfce504164e1507d5', 
'235896', 
'iDeal', 
'test' )

{'isSubmitted': False, 'detail': 'Over max submission count of Daily. 일일 제출 가능한 최대 횟수가 초과 되었습니다.'}
